In [ ]:
import json
import os
import pprint
import shutil
from collections import defaultdict, Counter
from library_dicom.dicom_processor.tools.folders import write_json_file
from library_dicom.dicom_processor.tools.cleaning_series import *


In [ ]:
merged_content = generate_merged_file("/media/salim/DD 2To/Json Ahl")
write_json_file("/media/salim/DD 2To/Json Ahl", 'merged_files', merged_content)

In [ ]:
series_ready = find_studies_with_two_series("/media/salim/DD 2To/Json Ahl/merged_files.json")


In [ ]:
root_destination = "/media/salim/DD 2To/AHL2011_Validated_DICOMS"
for seriesID in series_ready : 
    print(seriesID)
    source_path = series_ready[seriesID]['path']
    parentPatientID = series_ready[seriesID]['parentPatientID']
    parentStudyUID = series_ready[seriesID]['parentStudyUID']
    print( series_ready[seriesID]['path'] )
    print( series_ready[seriesID]['parentStudyUID'] )
    print( series_ready[seriesID]['parentPatientID'] )
    destination = root_destination+"/"+parentPatientID+"/"+parentStudyUID+"/"+seriesID
    os.makedirs(destination, exist_ok=True)
    shutil.move(source_path, destination)